In [1]:
import pandas as pd
import requests
import numpy as np
from splinter import Browser
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
pd.options.mode.chained_assignment = None  # default='warn'
from time import sleep
from selenium.common.exceptions import NoSuchElementException

In [2]:
df_player_bio = pd.DataFrame(columns=['Player_ID', 'Name', 'rel_link', 'Position'])

In [6]:
# Intitialze web driver
from selenium.webdriver.chrome.options import Options
options = Options()
options.binary_location = "C:\Program Files\Google\Chrome Beta\Application\chrome.exe"
driver = webdriver.Chrome(options=options)

driver.get('https://fantasydata.com/nfl/fantasy-football-leaders?season=2021&seasontype=1&scope=1&subscope=1&aggregatescope=1&range=3')

In [7]:
def load_total_table():

    # Load Player Ids into table
    element = driver.find_element(By.XPATH,'/html/body/div[6]/div/div[3]/div[2]/div/div[2]/div/div/div[2]/section/div[2]/div[3]/div[2]/div[2]/label[3]/label/span')
    element.click()
    sleep(1)
    
    # Select table length to max of 300
    element = driver.find_element(By.XPATH,'/html/body/div[6]/div/div[3]/div[2]/div/div[2]/div/div/div[2]/section/div[2]/div[3]/div[2]/div[2]/a[3]')
    element.click()
    sleep(1)
    
    # kill google pop up ad
    element = driver.find_element(By.XPATH, '//*[@id="mmt-3c7c8681-860c-4f5a-b02c-3798fabee9a6-sticky-close"]')
    element.click()
    sleep(1)

    # load all table data
    for i in range(10):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        element = driver.find_element(By.LINK_TEXT, 'LOAD MORE')
        sleep(3)
        element.click()
        sleep(3)

In [8]:
load_total_table()

In [9]:
# turn the page into a soup element
html = driver.page_source
page = soup(html, 'html.parser')

In [10]:
# find the player IDs and names and add to dataframe
# find player id
results = page.find_all('td', class_='player-id')
list_ids = [x.text for x in results]
df_player_bio['Player_ID'] = list_ids
# df_player_bio
# print(list_ids)

# find player name
results = page.find_all('td', class_='align-left')
list_name = [x.a.text for x in results]
df_player_bio['Name'] = list_name
# print(list_name)


# find relative player link
list_link = [x.a.get('href') for x in results]
df_player_bio['rel_link'] = list_link

# find position in order to filter
results = page.find_all('span', attrs={'ng-bind': 'dataItem.Position'})
list_pos = [x.text for x in results]
df_player_bio['Position'] = list_pos

df_player_bio.head()


,Player_ID,Name,rel_link,Position
0,19801,Josh Allen,/nfl/josh-allen-fantasy/19801,QB
1,21681,Justin Herbert,/nfl/justin-herbert-fantasy/21681,QB
2,4314,Tom Brady,/nfl/tom-brady-fantasy/4314,QB
3,18890,Patrick Mahomes,/nfl/patrick-mahomes-fantasy/18890,QB
4,21682,Jonathan Taylor,/nfl/jonathan-taylor-fantasy/21682,RB


In [11]:
positions = ['QB', 'RB', 'TE', 'WR']
df_player_bio = df_player_bio[df_player_bio['Position'].isin(positions)]
df_player_bio.sort_values('Name', ignore_index=True, inplace=True)
df_player_bio

,Player_ID,Name,rel_link,Position
0,21042,A.J. Brown,/nfl/aj-brown-fantasy/21042,WR
1,12845,A.J. Green,/nfl/aj-green-fantasy/12845,WR
2,21802,AJ Dillon,/nfl/aj-dillon-fantasy/21802,RB
3,21751,Aaron Fuller,/nfl/aaron-fuller-fantasy/21751,WR
4,19045,Aaron Jones,/nfl/aaron-jones-fantasy/19045,RB
...,...,...,...,...
661,20770,Zach Gentry,/nfl/zach-gentry-fantasy/20770,TE
662,19172,Zach Pascal,/nfl/zach-pascal-fantasy/19172,WR
663,22495,Zach Wilson,/nfl/zach-wilson-fantasy/22495,QB
664,21784,Zack Moss,/nfl/zack-moss-fantasy/21784,RB


create a for loop that goes to each url using https://fantasydata.com/nfl/{first_name}-{last_name}-fantasy/{player_id}
creates a soup element from that page, adds the data to the dataframe, then export moves to the next one

In [ ]:
url = r'https://fantasydata.com'
count = 0
total = len(df_player_bio)

for index, row in df_player_bio.iterrows():
    # testing loop break
#     if count >= 5:
#         break
#     count += 1
    
    if count != 0 and count % 50 == 0:
        print("***Break Time...***")
        sleep(10)
        

    
    # building search url
    player = row['Name']
    player_url = row['rel_link']
    search_url = url + player_url
    print(f'Searching for {player}')

    driver.get(search_url)
    print('Found...Adding Data...')
    
    # create soup object
    html = driver.page_source
    page = soup(html, 'html.parser')
    
    # looking for player data
    results = page.find('div', class_='player-info-dl')
    bio = results.find_all('dd')
    
    # grabbing data
    height_weight = bio[0].span.text
    dob_age = bio[1].span.text
    experience = bio[2].text
    draft = bio[3].text
    college = bio[4].text
    
    # adding to dataframe
    df_player_bio.loc[index,'height_weight'] = height_weight
    df_player_bio.loc[index, 'dob_age'] = dob_age
    df_player_bio.loc[index,'experience'] = experience
    df_player_bio.loc[index,'draft'] = draft
    df_player_bio.loc[index,'college'] = college
    print(f'Player Data Added: {player}')
    print('--------------------')
    sleep(2)
    

Searching for A.J. Brown
Found...Adding Data...
Player Data Added: A.J. Brown
--------------------
Searching for A.J. Green
Found...Adding Data...
Player Data Added: A.J. Green
--------------------
Searching for AJ Dillon
Found...Adding Data...
Player Data Added: AJ Dillon
--------------------
Searching for Aaron Fuller
Found...Adding Data...
Player Data Added: Aaron Fuller
--------------------
Searching for Aaron Jones
Found...Adding Data...
Player Data Added: Aaron Jones
--------------------
Searching for Aaron Parker
Found...Adding Data...
Player Data Added: Aaron Parker
--------------------
Searching for Aaron Rodgers
Found...Adding Data...
Player Data Added: Aaron Rodgers
--------------------
Searching for Adam Humphries
Found...Adding Data...
Player Data Added: Adam Humphries
--------------------
Searching for Adam Shaheen
Found...Adding Data...
Player Data Added: Adam Shaheen
--------------------
Searching for Adam Thielen
Found...Adding Data...
Player Data Added: Adam Thielen
-

Searching for Calvin Ridley
Found...Adding Data...
Player Data Added: Calvin Ridley
--------------------
Searching for Cam Akers
Found...Adding Data...
Player Data Added: Cam Akers
--------------------
Searching for Cam Newton
Found...Adding Data...
Player Data Added: Cam Newton
--------------------
Searching for Cam Sims
Found...Adding Data...
Player Data Added: Cam Sims
--------------------
Searching for Cameron Batson
Found...Adding Data...
Player Data Added: Cameron Batson
--------------------
Searching for Cameron Brate
Found...Adding Data...
Player Data Added: Cameron Brate
--------------------
Searching for Carlos Hyde
Found...Adding Data...
Player Data Added: Carlos Hyde
--------------------
Searching for Carson Wentz
Found...Adding Data...
Player Data Added: Carson Wentz
--------------------
Searching for Case Keenum
Found...Adding Data...
Player Data Added: Case Keenum
--------------------
Searching for Cedrick Wilson Jr.
Found...Adding Data...
Player Data Added: Cedrick Wils

Found...Adding Data...
Player Data Added: Darrel Williams
--------------------
Searching for Darrell Daniels
Found...Adding Data...
Player Data Added: Darrell Daniels
--------------------
Searching for Darrell Henderson Jr.
Found...Adding Data...
Player Data Added: Darrell Henderson Jr.
--------------------
Searching for Darren Fells
Found...Adding Data...
Player Data Added: Darren Fells
--------------------
Searching for Darren Waller
Found...Adding Data...
Player Data Added: Darren Waller
--------------------
Searching for Darrynton Evans
Found...Adding Data...
Player Data Added: Darrynton Evans
--------------------
Searching for Daurice Fountain
Found...Adding Data...
Player Data Added: Daurice Fountain
--------------------
Searching for Davante Adams
Found...Adding Data...
Player Data Added: Davante Adams
--------------------
Searching for David Blough
Found...Adding Data...
Player Data Added: David Blough
--------------------
Searching for David Johnson
Found...Adding Data...
Play

Searching for Foster Moreau
Found...Adding Data...
Player Data Added: Foster Moreau
--------------------
Searching for Frank Darby
Found...Adding Data...
Player Data Added: Frank Darby
--------------------
Searching for Freddie Swain
Found...Adding Data...
Player Data Added: Freddie Swain
--------------------
Searching for Gabriel Davis
Found...Adding Data...
Player Data Added: Gabriel Davis
--------------------
Searching for Gardner Minshew
Found...Adding Data...
Player Data Added: Gardner Minshew
--------------------
Searching for Garrett Gilbert
Found...Adding Data...
Player Data Added: Garrett Gilbert
--------------------
Searching for Garrett Griffin
Found...Adding Data...
Player Data Added: Garrett Griffin
--------------------
Searching for Gary Brightwell
Found...Adding Data...
Player Data Added: Gary Brightwell
--------------------
Searching for Geno Smith
Found...Adding Data...
Player Data Added: Geno Smith
--------------------
Searching for Geoff Swaim
Found...Adding Data...


In [ ]:
# exit the driver
driver.quit()

Clean the position bio data to their own columns

In [ ]:
df

export the dataframe to a csv file

In [ ]:
file_save = '../Resources/player_bio_data.csv'
df_player_bio.to_csv(file_save, index=False, na_rep='NULL')